In [2]:
import pandas as pd
from sqlalchemy import create_engine
import re

<h1>Importing relevant files</h1>

In [3]:
astro_file = "./ETL-Project-Data/astronauts.csv"
astro_data_df = pd.read_csv(astro_file)
astro_data_df.head()

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
0,Joseph M. Acaba,2004.0,19.0,Active,5/17/1967,"Inglewood, CA",Male,University of California-Santa Barbara; Univer...,Geology,Geology,NaN,NaN,2,3307,2,13.0,"STS-119 (Discovery), ISS-31/32 (Soyuz)",NaN,NaN
1,Loren W. Acton,NaN,NaN,Retired,3/7/1936,"Lewiston, MT",Male,Montana State University; University of Colorado,Engineering Physics,Solar Physics,NaN,NaN,1,190,0,0.0,STS 51-F (Challenger),NaN,NaN
2,James C. Adamson,1984.0,10.0,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN
3,Thomas D. Akers,1987.0,12.0,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN
4,Buzz Aldrin,1963.0,3.0,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN


In [7]:
space_file = "./ETL-Project-Data/Space_Corrected.csv"
space_data_df = pd.read_csv(space_file)
space_data_df.head()

,Unnamed: 0,Unnamed: 0.1,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


<h1>Separated "Missions" into unique columns by commas </h1>

In [8]:
new_astro_df = astro_data_df[["Name", "Birth Date", "Missions", "Space Flight (hr)", "Space Walks (hr)"]].copy()

new_astro_df['Missions'] = new_astro_df['Missions'].str.replace(r"\(.*\)","")
new_astro_df['Missions'] = new_astro_df['Missions'].str.replace(' ', '')


new_astro_df = pd.concat([new_astro_df, new_astro_df["Missions"].str.split(',', expand=True)], axis=1)
new_astro_df.head()


,Name,Birth Date,Missions,Space Flight (hr),Space Walks (hr),0,1,2,3,4
0,Joseph M. Acaba,5/17/1967,STS-119,3307,13.0,STS-119,None,None,None,None
1,Loren W. Acton,3/7/1936,STS51-F,190,0.0,STS51-F,None,None,None,None
2,James C. Adamson,3/3/1946,STS-28,334,0.0,STS-28,None,None,None,None
3,Thomas D. Akers,5/20/1951,STS-41,814,29.0,STS-41,None,None,None,None
4,Buzz Aldrin,1/20/1930,"Gemini12,Apollo11",289,8.0,Gemini12,Apollo11,None,None,None


<h1>Seperated "Detail" into unique columns by pipe character </h1>

In [12]:
new_space_df = space_data_df[["Company Name", "Location", "Datum", "Detail", " Rocket", "Status Mission"]].copy()
new_space_df['Detail'] = [x.split(" | ")[1] for x in new_space_df['Detail']]
new_space_df['Detail'] = new_space_df['Detail'].str.replace(' ', '')
new_space_df = pd.concat([new_space_df, new_space_df["Detail"].str.split('&', expand=True)], axis=1)



In [13]:
#Rename columns
new_astro_df=new_astro_df.rename(columns={ 0: 'Mission 1',
                                           1: 'Mission 2',
                                           2: 'Mission 3',
                                           3: 'Mission 4',
                                           4: 'Mission 5'
                                         })

In [16]:
#Rename columns
new_space_df = new_space_df.rename(columns={ 0: 'Rocket 1',
                                           1: 'Rocket 2',
                                           2: 'Rocket 3',
                                         })


<h1>Editting the added columns to later concatenate</h1>

In [17]:
#For first CSV file imported
mission1 = new_astro_df[['Name', 'Birth Date', 'Space Flight (hr)', 'Space Walks (hr)', 'Mission 1']]

In [18]:
mission1 = mission1.rename(columns={'Mission 1': 'Mission'})

In [19]:
mission2 = new_astro_df[['Name', 'Birth Date', 'Space Flight (hr)', 'Space Walks (hr)', 'Mission 2']]

In [20]:
mission2 = mission2.dropna(how='any').rename(columns={'Mission 2': 'Mission'})

In [21]:
mission3 = new_astro_df[['Name', 'Birth Date', 'Space Flight (hr)', 'Space Walks (hr)', 'Mission 3']]
mission3 = mission3.dropna(how='any').rename(columns={'Mission 3': 'Mission'})

In [22]:
mission4 = new_astro_df[['Name', 'Birth Date', 'Space Flight (hr)', 'Space Walks (hr)', 'Mission 4']]
mission4 = mission4.dropna(how='any').rename(columns={'Mission 4': 'Mission'})

In [23]:
mission5 = new_astro_df[['Name', 'Birth Date', 'Space Flight (hr)', 'Space Walks (hr)', 'Mission 5']]
mission5 = mission5.dropna(how='any').rename(columns={'Mission 5': 'Mission'})

<h1>Concatenation of the 5 new dataframes</h1>

In [31]:
a = pd.concat([mission1, mission2])

In [32]:
b = pd.concat([a, mission3])

In [33]:
c = pd.concat([b, mission3])

In [34]:
d = pd.concat([c, mission4])

In [35]:
astro_final = pd.concat([d, mission5])

In [36]:
astro_final

,Name,Birth Date,Space Flight (hr),Space Walks (hr),Mission
0,Joseph M. Acaba,5/17/1967,3307,13.0,STS-119
1,Loren W. Acton,3/7/1936,190,0.0,STS51-F
2,James C. Adamson,3/3/1946,334,0.0,STS-28
3,Thomas D. Akers,5/20/1951,814,29.0,STS-41
4,Buzz Aldrin,1/20/1930,289,8.0,Gemini12
...,...,...,...,...,...
78,Charles Conrad Jr.,5/2/1930,1179,12.0,Skylab2
208,James A. Lovell Jr.,3/25/1928,715,0.0,Apollo13
305,Thomas P. Stafford,9/17/1930,507,0.0,Apollo-SoyuzTestProject
355,John W. Young,9/24/1930,835,20.0,Apollo16


<h1>Editting the added columns to later concatenate</h1>

In [38]:
#For second csv file imported
Rocket1 = new_space_df[['Company Name', 'Location',	'Datum', ' Rocket', 'Status Mission','Rocket 1']]
Rocket1 = Rocket1.dropna(how='any').rename(columns={'Rocket 1': 'Rocket'})
Rocket1

,Company Name,Location,Datum,Rocket,Status Mission,Rocket
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",50.0,Success,StarlinkV1L9
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",29.75,Success,Gaofen-904
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",65.0,Success,Ekspress-80
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",145.0,Success,Perseverance
5,CASC,"LC-9, Taiyuan Satellite Launch Center, China","Sat Jul 25, 2020 03:13 UTC",64.68,Success,"Ziyuan-303,Apocalypse-10"
...,...,...,...,...,...,...
3855,US Air Force,"SLC-4W, Vandenberg AFB, California, USA","Fri Jul 29, 1966 18:43 UTC",59.0,Success,KH-8
3971,US Air Force,"SLC-20, Cape Canaveral AFS, Florida, USA","Thu May 06, 1965 15:00 UTC",63.23,Success,LES2
3993,US Air Force,"SLC-20, Cape Canaveral AFS, Florida, USA","Thu Feb 11, 1965 15:19 UTC",63.23,Success,LES1
4000,US Air Force,"SLC-20, Cape Canaveral AFS, Florida, USA","Thu Dec 10, 1964 16:52 UTC",63.23,Success,Transtage2


In [39]:
Rocket2 = new_space_df[['Company Name', 'Location',	'Datum', ' Rocket', 'Status Mission','Rocket 2']]
Rocket2 = Rocket2.dropna(how='any').rename(columns={'Rocket 2': 'Rocket'})
Rocket2

,Company Name,Location,Datum,Rocket,Status Mission,Rocket
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",50.0,Success,BlackSky
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",29.75,Success,Q-SAT
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",65.0,Success,Ekspress-103
5,CASC,"LC-9, Taiyuan Satellite Launch Center, China","Sat Jul 25, 2020 03:13 UTC",64.68,Success,NJU-HKU1
16,CASC,"LC-9, Taiyuan Satellite Launch Center, China","Fri Jul 03, 2020 03:10 UTC",64.68,Success,BY-02
...,...,...,...,...,...,...
1515,Northrop,"Stargazer, Vandenberg AFB, California, USA","Mon Apr 03, 1995 13:48 UTC",40.0,Success,OrbView-1
1582,Northrop,"SLC-576E, Vandenberg AFB, California, USA","Sun Mar 13, 1994 22:32 UTC",45.0,Success,DARPASAT
1831,Northrop,"NB-52B Carrier, Edwards AFB, California, USA","Thu Apr 05, 1990 19:10 UTC",40.0,Success,NavySat
1855,Martin Marietta,"SLC-40, Cape Canaveral AFS, Florida, USA","Mon Jan 01, 1990 00:07 UTC",136.6,Success,JCSAT-2


In [40]:
Rocket3 = new_space_df[['Company Name', 'Location',	'Datum', ' Rocket', 'Status Mission','Rocket 3']]
Rocket3 = Rocket3.dropna(how='any').rename(columns={'Rocket 3': 'Rocket'})
Rocket3

,Company Name,Location,Datum,Rocket,Status Mission,Rocket
131,Arianespace,"ELA-3, Guiana Space Centre, French Guiana, France","Thu Jun 20, 2019 21:43 UTC",200.0,Success,TT-16
634,ISRO,"First Launch Pad, Satish Dhawan Space Centre, ...","Sun Sep 09, 2012 04:23 UTC",21.0,Success,PROITERES
690,ISRO,"First Launch Pad, Satish Dhawan Space Centre, ...","Wed Apr 20, 2011 04:42 UTC",25.0,Success,YouthSat
886,ISRO,"First Launch Pad, Satish Dhawan Space Centre, ...","Wed Jan 10, 2007 03:54 UTC",25.0,Success,Rideshares


<h1>Concatenation of the 3 new dataframes</h1>

In [41]:
a = pd.concat([Rocket1, Rocket2])

In [42]:
rocket_final = pd.concat([a, Rocket3])

In [43]:
rocket_final

,Company Name,Location,Datum,Rocket,Status Mission,Rocket
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",50.0,Success,StarlinkV1L9
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",29.75,Success,Gaofen-904
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",65.0,Success,Ekspress-80
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",145.0,Success,Perseverance
5,CASC,"LC-9, Taiyuan Satellite Launch Center, China","Sat Jul 25, 2020 03:13 UTC",64.68,Success,"Ziyuan-303,Apocalypse-10"
...,...,...,...,...,...,...
3971,US Air Force,"SLC-20, Cape Canaveral AFS, Florida, USA","Thu May 06, 1965 15:00 UTC",63.23,Success,LCS1
131,Arianespace,"ELA-3, Guiana Space Centre, French Guiana, France","Thu Jun 20, 2019 21:43 UTC",200.0,Success,TT-16
634,ISRO,"First Launch Pad, Satish Dhawan Space Centre, ...","Sun Sep 09, 2012 04:23 UTC",21.0,Success,PROITERES
690,ISRO,"First Launch Pad, Satish Dhawan Space Centre, ...","Wed Apr 20, 2011 04:42 UTC",25.0,Success,YouthSat


In [44]:
astro_final

,Name,Birth Date,Space Flight (hr),Space Walks (hr),Mission
0,Joseph M. Acaba,5/17/1967,3307,13.0,STS-119
1,Loren W. Acton,3/7/1936,190,0.0,STS51-F
2,James C. Adamson,3/3/1946,334,0.0,STS-28
3,Thomas D. Akers,5/20/1951,814,29.0,STS-41
4,Buzz Aldrin,1/20/1930,289,8.0,Gemini12
...,...,...,...,...,...
78,Charles Conrad Jr.,5/2/1930,1179,12.0,Skylab2
208,James A. Lovell Jr.,3/25/1928,715,0.0,Apollo13
305,Thomas P. Stafford,9/17/1930,507,0.0,Apollo-SoyuzTestProject
355,John W. Young,9/24/1930,835,20.0,Apollo16


In [45]:
#exporting final dataframes to CSV files
astro_final.to_csv('astro_final.csv', index=False)
rocket_final.to_csv('rocket_final.csv', index=False)